Why do we need macro variables in our dataset?

Macroeconomic variables provide essential context that helps the model interpret firm-level financial data across different years. They capture underlying economic conditions — such as inflation, interest rates, or GDP growth — which can influence a company’s performance but are not directly visible in its financial statements.

For example, two companies with similar financial ratios may experience very different revenue growth outcomes if one operated during a post-COVID recovery (2021) and the other during a period of rising interest rates and geopolitical uncertainty (2022–2023). Macroeconomic indicators encode this latent information about the external environment — including events like the pandemic, war, or economic crisis — allowing the model to generalize better across time and conditions.

In [7]:
import pandas as pd
import numpy as np
from typing import List
import requests
import warnings

In [8]:
df_all_exp_cat_cl = pd.read_csv("Step3_all_companies_features_with_labels_cleaned.csv")

In [9]:
df_all_exp_cat_cl.head()


,Date,revenue,netincome,r&d,sg&a,ebit,revenuegrowth,ebitmargin,netmargin,rd_to_revenue,...,roe,liabilities_to_equity,debt_to_equity,nopat,roic,sales_to_capital,fcf_to_netincome,revenuegrowth_t+1,ticker,label
0,2019-01-27,11716000000,4141000000,2376000000,991000000,3804000000,0.206094,0.324684,0.353448,0.202800,...,0.443267,0.422822,0.212802,3.005160e+09,0.284903,3.336667,0.758995,-0.068112,NVDA,Mega‑Cap Tech
1,2019-01-31,177722000,-53885000,58841000,140116000,-55084000,0.597458,-0.309945,-0.303198,0.331085,...,-0.322680,0.848855,0.035523,-4.351636e+07,1.083548,7.567054,0.215997,0.524189,SMAR,Collab & CX SaaS
2,2019-01-31,249824000,-140077000,84551000,214899000,-136864000,1.103746,-0.547842,-0.560703,0.338442,...,-1.876299,4.802869,0.000000,-1.081226e+08,7.862315,3.073561,0.468407,0.927009,CRWD,Cloud & Data
3,2019-01-31,267016000,-99011000,89854000,201359000,-97765000,0.728046,-0.366139,-0.370805,0.336512,...,-0.374239,1.772374,0.819674,-7.723435e+07,-0.231523,16.427716,0.493248,0.579381,MDB,Cloud & Data
4,2019-01-31,2822180000,-418258000,1211832000,1238682000,-463284000,0.316899,-0.164158,-0.148204,0.429396,...,-0.213567,1.818948,0.615172,-3.659944e+08,-0.144967,3.189048,-0.941287,0.285250,WDAY,Enterprise SaaS


What Macroeconomic Data Makes can we use?

- Real GDP (YoY): Captures overall economic growth (code: GDPC1) ;
  - It represents the inflation-adjusted size of all final goods & services the U.S. economy would produce in a year if the quarter’s pace persisted for four quarters;
  - The Q4 reading is what analysts quote as “GDP for choosen financial year”;
- Consumer Price Index: Measures inflation — affects pricing and margins (code: CPIAUCSL);
- Federal Funds Rate: Affects cost of capital and investment decisions (code: FEDFUNDS);
- Unemployment Rate: Proxy for labor market and demand environment (code: UNRATE);
- 10-Year Treasury Yield: Benchmark for cost of debt (code: GS10);
- Consumer Sentiment Index: Gauges consumer outlook and demand expectations (code: UMCSENT).

If a company’s fiscal year ends in January or early February, almost all of the revenues, costs, and operating environment for that “fiscal‑year” actually happened in the previous calendar year. When we want to line the financial row up with a single calendar‑year macro datapoint (GDP, CPI, unemployment, etc.), using the prior calendar year is the cleanest one‑line rule.

FRED (and most statistical agencies) aggregate macro data on a Jan → Dec calendar:

- GDP 2022 = average/total of 2022‑Q1 … 2022‑Q4;
- CPI 2022 = mean of monthly CPI prints from Jan‑22 to Dec‑22.

Many companies (including tech firms like Adobe, NVIDIA, Salesforce) have fiscal years ending in January or February, which means:
- Most of the activity (revenues, costs, macro exposure) occurred in the previous calendar year;
- If you align fiscal data ending in Jan–Feb to the same calendar year’s GDP, CPI, or interest rates, you are incorrectly incorporating future macro events into the financial inputs. This causes look-ahead bias — and that's especially problematic for any predictive modeling or backtesting.

In summary:
- January 27, 2019 → macro_year = 2018;
- March 31, 2019 → macro_year = 2019.

This adjustment ensures each company’s fiscal-year financials are correctly linked to the calendar-year macroeconomic conditions that actually influenced them.

In [10]:
# This step is essential to the merge of two dfatasets (Ensure 'macro_year' exists in both datasets)

df_all_exp_cat_cl["Date"] = pd.to_datetime(df_all_exp_cat_cl["Date"])          # if it isn’t already
df_all_exp_cat_cl["macro_year"] = np.where(
    df_all_exp_cat_cl["Date"].dt.month > 2,                    # Mar–Dec → same calendar year
    df_all_exp_cat_cl["Date"].dt.year,
    df_all_exp_cat_cl["Date"].dt.year - 1                      # Jan–Feb → previous calendar year
)

In [11]:
df_all_exp_cat_cl.head()

,Date,revenue,netincome,r&d,sg&a,ebit,revenuegrowth,ebitmargin,netmargin,rd_to_revenue,...,liabilities_to_equity,debt_to_equity,nopat,roic,sales_to_capital,fcf_to_netincome,revenuegrowth_t+1,ticker,label,macro_year
0,2019-01-27,11716000000,4141000000,2376000000,991000000,3804000000,0.206094,0.324684,0.353448,0.202800,...,0.422822,0.212802,3.005160e+09,0.284903,3.336667,0.758995,-0.068112,NVDA,Mega‑Cap Tech,2018
1,2019-01-31,177722000,-53885000,58841000,140116000,-55084000,0.597458,-0.309945,-0.303198,0.331085,...,0.848855,0.035523,-4.351636e+07,1.083548,7.567054,0.215997,0.524189,SMAR,Collab & CX SaaS,2018
2,2019-01-31,249824000,-140077000,84551000,214899000,-136864000,1.103746,-0.547842,-0.560703,0.338442,...,4.802869,0.000000,-1.081226e+08,7.862315,3.073561,0.468407,0.927009,CRWD,Cloud & Data,2018
3,2019-01-31,267016000,-99011000,89854000,201359000,-97765000,0.728046,-0.366139,-0.370805,0.336512,...,1.772374,0.819674,-7.723435e+07,-0.231523,16.427716,0.493248,0.579381,MDB,Cloud & Data,2018
4,2019-01-31,2822180000,-418258000,1211832000,1238682000,-463284000,0.316899,-0.164158,-0.148204,0.429396,...,1.818948,0.615172,-3.659944e+08,-0.144967,3.189048,-0.941287,0.285250,WDAY,Enterprise SaaS,2018


In [12]:
# Start fetching from FRED

In [13]:
import pandas as pd
import numpy as np
import requests # to call the FRED API and download JSON data
from functools import reduce # to merge multiple DataFrames in a loop-like manner.

# Function 1: Fetch data from FRED API

def fetch_fred_series(series_id: str, api_key: str, start_date: str = "2010-01-01") -> pd.DataFrame:
    """
    Download a time series from the FRED API and return it as a pandas DataFrame.
    
    Parameters:
    -----------
    series_id : str
        The unique ID of the macroeconomic series on FRED (e.g., 'GDPC1' for real GDP).
    api_key : str
        Your personal API key from https://fred.stlouisfed.org.
    start_date : str
        The earliest date to fetch data from (format: 'YYYY-MM-DD').
        
    Returns:
    --------
    pd.DataFrame
        A DataFrame with columns:
            - 'date': timestamps of observations
            - '<series_id>': numeric values (converted from strings)
    """
    # FRED has many endpoints — this one gives you historical time-series values for a single macroeconomic indicator.
    url = "https://api.stlouisfed.org/fred/series/observations"

    # This dictionary contains your query parameters:
    # series_id: e.g., "GDPC1" (GDP), "CPIAUCSL" (inflation).
    # api_key: the private key from FRED.
    # file_type: return JSON.
    # observation_start: filter for data starting in 2010 or later.
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json",
        "observation_start": start_date
    }

    # Make HTTP request. Performs the actual API call.
    response = requests.get(url, params=params)
    
    response.raise_for_status()  # Raise error if the request fails

    # Extract observation data from the JSON response
    data = response.json()
    observations = data["observations"]

    # Build a clean DataFrame from FRED response
    df = pd.DataFrame(observations)
    # Converts string date to datetime
    df["date"] = pd.to_datetime(df["date"])
    # Converts string "value" to float, safely ignoring "." or "NaN" strings
    df[series_id] = pd.to_numeric(df["value"], errors="coerce")

    # Returns a clean DataFrame with just the date and value.
    return df[["date", series_id]]

In [14]:
def aggregate_macro_series(df: pd.DataFrame, series_id: str) -> pd.DataFrame:
    """
    Resample a macroeconomic time series to annual frequency, using best practice aggregation:
    - For 'GDPC1' (GDP), take the last available quarterly observation of the year (Q4).
    - For other series, take the average value over the year.

    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing a 'date' index and a macroeconomic series.
    series_id : str
        The FRED series ID to guide the resampling strategy.

    Returns:
    --------
    pd.DataFrame
        A DataFrame with one row per calendar year and columns: ['macro_year', <series_id>].
    """
    # resample() only works with time-indexed DataFrames, so we set "date" as the index.
    # resample() is for time-based grouping — like groupby() but for time.
    df = df.set_index("date")

    # .resample("YE") groups the data into calendar years ending in December.
    # GDP is annualized already, so we use .last();
    # Other monthly indicators are noisy, so .mean() gives stability.

    if series_id == "GDPC1":
        # "last" picked the last value in each year. This is useful when using GDP (which is already annualized) — you don’t want to average quarterly GDP
        df_yearly = df.resample("YE").last()  # use Q4 GDP as the year-end state
    else:
        df_yearly = df.resample("YE").mean()  # average over year (e.g., CPI, FEDFUNDS)

    # Reset the index to turn date back into a column;
    # Extract year from the date (which will be Dec 31) and call it macro_year
    # Drop the unnecessary full date.

    df_yearly = df_yearly.reset_index()
    df_yearly["macro_year"] = df_yearly["date"].dt.year
    df_yearly = df_yearly.drop(columns=["date"])

    return df_yearly

In [15]:
# Execution Starts Here 

# Define the macroeconomic series to fetch
# This dictionary gives an alias → description map for each FRED series_id
fred_series = {
    "GDPC1": "Real GDP (SAAR, Bil. 2012$)",
    "CPIAUCSL": "Consumer Price Index (CPI)",
    "FEDFUNDS": "Federal Funds Rate",
    "UNRATE": "Unemployment Rate",
    "GS10": "10-Year Treasury Yield",
    "UMCSENT": "Consumer Sentiment Index"
}

# Your FRED API key
# fred_api_key = ""
import os
load_dotenv()  # Automatically loads from .env in current directory
fred_api_key = os.getenv("FRED_API_KEY")

# Step 1: Fetch all macro series from FRED API
# Creates a dictionary where:
# key = macro name (e.g. "GDPC1"),
# value = the fetched DataFrame.
macro_data = {}
for series_id in fred_series:
    print(f"Fetching {series_id}...")
    macro_data[series_id] = fetch_fred_series(series_id, fred_api_key)

Fetching GDPC1...
Fetching CPIAUCSL...
Fetching FEDFUNDS...
Fetching UNRATE...
Fetching GS10...
Fetching UMCSENT...


In [16]:
# macro_data

The aggregation of macro data to Annual Format

Only take the mean when the series is really higher-frequency (daily, monthly, quarterly).
If a series already has exactly one observation per year (e.g., sits on 31 Dec), averaging after resampling just combines a bunch of identical copies and gives you the same number again—wasting compute and sometimes creating duplicate rows when you merge frames.

Should you use .mean() or .last() dor GDP?

Use .last() for GDP series like GDPC1 if you want to reflect the economy's state at the end of the year:
- GDP is annualized by construction (SAAR), so each point already reflects an annual pace.;
- Taking the mean of Q1–Q4 would blur seasonal movements and may misrepresent the most recent state of the economy;
- Using the last quarter (usually Q4, reported in Dec) aligns better with end-of-year conditions — especially if you’re trying to predict future growth.



In [17]:
# Step 2: Aggregate each series to calendar-year format using best practice
# Processes each time series with the right logic (mean() or last()), and collects them into a list.

macro_yearly_data = []
for series_id, df in macro_data.items():
    yearly_df = aggregate_macro_series(df, series_id)
    macro_yearly_data.append(yearly_df)

In [18]:
# macro_yearly_data

In [19]:
# Step 3: Merge all macro series into one DataFrame on 'macro_year'
# use of reduce() to repeatedly apply pd.merge() across all DataFrames in macro_yearly_data
# reduce() is recursive - takes a list of items and repeatedly applies a function pairwise to combine them into a single result.
# Outer: Keep all years from all DataFrames. If one series is missing for a year, just fill it with NaN;
# We could also use: 
# "inner" → keeps only years present in all series (risky — more NaN loss).;
# "left" or "right" → use one dataset as anchor.
# lambda is assigning: left → the running merged result so far, left → the running merged result so far

macro_merged = reduce(
    lambda left, right: pd.merge(left, right, on="macro_year", how="outer"),
    macro_yearly_data
)

# Step 4: Sort for readability and reset index
macro_merged = macro_merged.sort_values("macro_year").reset_index(drop=True)

# Show the final macro dataset (one row per year)
macro_merged.tail()

,GDPC1,macro_year,CPIAUCSL,FEDFUNDS,UNRATE,GS10,UMCSENT
11,21960.388,2021,270.967917,0.080000,5.358333,1.442500,77.616667
12,22249.459,2022,292.625417,1.683333,3.641667,2.951667,58.975000
13,22960.600,2023,304.704167,5.024167,3.633333,3.957500,65.350000
14,23542.349,2024,313.697833,5.143333,4.033333,4.208333,72.541667
15,23685.287,2025,320.146167,4.330000,4.133333,4.406667,59.560000


In [20]:
df_final = df_all_exp_cat_cl.merge(macro_merged, on="macro_year", how="left")
# or df_final = pd.merge(df_all_exp_cat_cl, macro_merged, on="macro_year", how="left")

In [21]:
df_final

,Date,revenue,netincome,r&d,sg&a,ebit,revenuegrowth,ebitmargin,netmargin,rd_to_revenue,...,revenuegrowth_t+1,ticker,label,macro_year,GDPC1,CPIAUCSL,FEDFUNDS,UNRATE,GS10,UMCSENT
0,2019-01-27,11716000000,4141000000,2376000000,991000000,3804000000,0.206094,0.324684,0.353448,0.202800,...,-0.068112,NVDA,Mega‑Cap Tech,2018,20304.874,251.099500,1.831667,3.891667,2.910000,98.366667
1,2019-01-31,177722000,-53885000,58841000,140116000,-55084000,0.597458,-0.309945,-0.303198,0.331085,...,0.524189,SMAR,Collab & CX SaaS,2018,20304.874,251.099500,1.831667,3.891667,2.910000,98.366667
2,2019-01-31,249824000,-140077000,84551000,214899000,-136864000,1.103746,-0.547842,-0.560703,0.338442,...,0.927009,CRWD,Cloud & Data,2018,20304.874,251.099500,1.831667,3.891667,2.910000,98.366667
3,2019-01-31,267016000,-99011000,89854000,201359000,-97765000,0.728046,-0.366139,-0.370805,0.336512,...,0.579381,MDB,Cloud & Data,2018,20304.874,251.099500,1.831667,3.891667,2.910000,98.366667
4,2019-01-31,2822180000,-418258000,1211832000,1238682000,-463284000,0.316899,-0.164158,-0.148204,0.429396,...,0.285250,WDAY,Enterprise SaaS,2018,20304.874,251.099500,1.831667,3.891667,2.910000,98.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,2024-03-31,475422000,147582000,81983000,170956000,163878000,0.134518,0.344700,0.310423,0.172443,...,0.199774,DOCS,Vertical SaaS,2024,23542.349,313.697833,5.143333,4.033333,4.208333,72.541667
490,2024-04-30,310582000,-279696000,201365000,295537000,-318342000,0.164122,-1.024985,-0.900554,0.648347,...,0.252668,AI,AI‑DevOps,2024,23542.349,313.697833,5.143333,4.033333,4.208333,72.541667
491,2024-04-30,1267321000,61720000,341951000,720276000,-129902000,0.185532,-0.102501,0.048701,0.269822,...,0.170419,ESTC,Cloud & Data,2024,23542.349,313.697833,5.143333,4.033333,4.208333,72.541667
492,2024-05-31,52961000000,10467000000,8915000000,9822000000,15353000000,0.060195,0.289893,0.197636,0.168331,...,0.083798,ORCL,Mature Pivots,2024,23542.349,313.697833,5.143333,4.033333,4.208333,72.541667


In [22]:
df_final.set_index("Date", inplace=True)
df_final.to_csv("Step4_all_companies_features_with_labels_cleaned_macro.csv")